# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import os

# Define Source

In [2]:
link = 'https://autismcanada.org/autism-explained/cooccurring-conditions/'

# Read 100 questions list

In [3]:
file = 'Excel/' + os.listdir('Excel')[-1]
df = pd.read_excel(file,engine='openpyxl')

df[link]=np.nan
df.head()

,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/,https://www.autism.org/symptoms-of-autism/,https://www.autism.org/autism-prognosis/,https://autismcanada.org/autism-explained/,https://autismcanada.org/autism-explained/cooccurring-conditions/
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN,NaN,NaN,NaN,NaN
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,Possible symptoms of autism may be observed by...,NaN,NaN,https://autismcanada.org/autism-explained/diag...,NaN
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN,NaN,NaN,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,Autism develops sometime during pregnancy and ...,NaN,NaN,NaN,NaN
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,Autism is a developmental disability that typi...,NaN,Autism is a neurodevelopmental disorder that b...,"Autism Spectrum Disorder, or autism, is a neur...",NaN


# Scrape QA pairs from website

In [4]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [5]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [6]:
x = soup.find('div',{'class':'elementor-text-editor elementor-clearfix'}).text
x

'\nCo-occurring ConditionsThere are several medical conditions that have been shown to be significantly more prevalent in those on the spectrum\xa0than the general population.Autism is a whole body disorder diagnosed by what is observable, however, these signs and symptoms are often the result of underlying co-occurring conditions.\xa0 Knowing what to look for is the first step in getting the most appropriate treatment.Some common co-occurring conditions found in those with autism:Autoimmune conditionsSeizure disordersOesophagitisGastritisRefluxAsthmaEczemaAllergiesEar infectionsRespiratory infectionsMigraine headachesAllergy disorder (including non IgE-mediated disorders or food intolerances)At times, it may be difficult for a health care practitioner to properly assess a patient with an ASD.\xa0 The parent or caregiver may not have known to watch for certain signs because they were told that it was “just the autism”.\xa0 There may be communication barriers or challenging behaviours p

In [10]:
l1 = [soup.find('h1').text]
k = [x]

# Check which questions are similar

In [11]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('autism spectrum disorder','asd',sent) # Compress term
    sent = re.sub('autism','asd',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [12]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [13]:
def sim(li):
    for c1,i in enumerate(li):
        i2 = clean_text(i)
        list1 = i2.split()

        temp1 = 0.0
        temp2 = ''
        temp3 = ''
        temp4 = 0
        temp5 = 0

        for c2,j in enumerate(df['Question']):
            j2 = clean_text(j)
            list2 = j2.split()

            sim = jaccard_similarity(list1, list2)

            if(sim>temp1):
                temp1 = sim
                temp2 = j
                temp3 = i
                temp4 = c2
                temp5 = c1

        if(temp1>0):
            print('Website --> ',temp3,'(Index {})'.format(temp5))
            print('100 questions list --> ',temp2,'(Index {})'.format(temp4))
            print(temp1)
            print('*'*100)

In [14]:
sim(l1)

Website -->  Co-occurring Conditions (Index 0)
100 questions list -->  Conditions that can occur with autism (Index 28)
0.25
****************************************************************************************************


In [15]:
# Website, 100Q
# 0, 28

In [16]:
df.iloc[28,-1]=k[0]

# Save Output

In [17]:
df.to_excel('Excel/Source3_2.xlsx',index=False)